In [67]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder


In [68]:
training_path = "dataset_lanl/training_final.csv"
testing_path = "dataset_lanl/testing_final.csv"

In [69]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_len):
        self.seq_len = seq_len
        self.data = data
        self.samples = self.create_sequences(data, seq_len)

    def create_sequences(self, data, seq_len):
        sequences = []
        for i in range(len(data) - seq_len):
            seq = data[i:i+seq_len]
            sequences.append(seq)
        return sequences

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return torch.tensor(self.samples[idx], dtype=torch.float32)


In [70]:
def load_csv_as_tensor(path, seq_len):
    # Keep it during scaling and modeling
    df = pd.read_csv(path)
    #split_index = int(len(df) * 0.3)
    #df = df.iloc[:split_index]
    df = df.drop(columns=["Unnamed: 0"], errors="ignore")

    # Label encode object columns
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = LabelEncoder().fit_transform(df[col].astype(str))

    # Normalize including 'time'
    scaler = MinMaxScaler()
    features = scaler.fit_transform(df.values)


    return TimeSeriesDataset(features, seq_len), scaler


In [71]:
SEQ_LEN = 30
BATCH_SIZE = 64

train_dataset, scaler = load_csv_as_tensor(training_path, SEQ_LEN)
test_dataset, _ = load_csv_as_tensor(testing_path, SEQ_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [72]:
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super().__init__()
        self.encoder = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.latent = nn.LSTM(hidden_dim, latent_dim, batch_first=True)
        self.decoder = nn.LSTM(latent_dim, hidden_dim, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        enc_out, _ = self.encoder(x)
        lat_out, _ = self.latent(enc_out)
        dec_out, _ = self.decoder(lat_out)
        out = self.output_layer(dec_out)
        return out


In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {device}")

model = LSTMAutoencoder(input_dim=train_dataset[0].shape[1], hidden_dim=64, latent_dim=32).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 5
model.train()

for epoch in range(EPOCHS):
    epoch_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        output = model(batch)
        loss = criterion(output, batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss / len(train_loader):.6f}")


Running on cpu
Epoch 1/5, Loss: 0.001408


KeyboardInterrupt: 

In [ ]:
model.eval()
errors = []
all_outputs = []

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        output = model(batch)
        loss = torch.mean((output - batch) ** 2, dim=(1,2))  # MSE per sequence
        errors.extend(loss.cpu().numpy())
        all_outputs.extend(output.cpu().numpy())


In [ ]:
errors = np.array(errors)
threshold = np.percentile(errors, 95)  # Can be tuned

anomalies = (errors > threshold).astype(int)
print(f"Anomalies detected: {np.sum(anomalies)} out of {len(anomalies)} sequences, {np.sum(anomalies)/len(anomalies):.3%}")

# Log anomalies (optional)
anomaly_indices = np.where(anomalies == 1)[0]
with open("anomaly_log.txt", "w") as f:
    for i in anomaly_indices:
        f.write(f"Anomaly at sequence starting index: {i}\n")


Anomalies detected: 9647 out of 192927 sequences, 5.000%


In [ ]:
torch.save(model.state_dict(), "lstm_autoencoder.pth")